<a href="https://colab.research.google.com/github/GrzegorzAndrzejczak/Programowanie-w-Pythonie/blob/main/notebooks/F%26B-pr%C3%B3bkowanie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Frequentism and Bayesianism -- David Barber (2020), Robert Casella oraz Jake VanderPlas (jakevdp)


$\newcommand{\N}{\mathbb{N}}
\newcommand{\R}{\mathbb{R}}
\newcommand{\PP}{\mathbb{P}}
\newcommand{\fB}{\mathfrak{B}}
\newcommand{\fF}{\mathfrak{F}}
\newcommand{\cX}{\mathcal{X}}
$


Ustalamy parametry startowe
### Przykład

In [ ]:
import numpy as np
from scipy import stats

np.random.seed(42)  # for reproducibility
N = 100             # more samples for the more complicated model
mu_true, sigma_true = 1000, 15    # stochastic flux model

F_true = stats.norm(mu_true, sigma_true).rvs(N)  # (unknown) true flux
F = stats.poisson(F_true).rvs()   # observed flux: true flux plus Poisson errors.
e = np.sqrt(F)  # root-N error, as above

def log_prior0(theta):
  # sigma needs to be positive. Czy wiemy coś jeszcze?
  if theta[1] <= 0:
    return -np.inf
  else:
    return 1/(1+theta[1])   # raczej 0 (rozkład jednostajny dla sigma>0), albo: np.log(theta[1]) - raczej NIE 0

def log_likelihood(theta, F, e):
  return -0.5 * np.sum(np.log(2 * np.pi * (theta[1] ** 2 + e ** 2))
                         + (F - theta[0]) ** 2 / (theta[1] ** 2 + e ** 2))

def log_posterior0(theta):
  return log_prior0(theta) + log_likelihood(theta, F, e)

ndim = 2        # number of parameters in the model
nwalkers = 100   # number of MCMC walkers
nsteps, nburn = 2000, 1000


def compute_log_prob0(coords):
  """Calculate the vector of log-probability for the walkers
  Args:
      coords: (ndarray[..., ndim]) The position vector in parameter
          space where the probability should be calculated.
  """
  log_prob = np.array([log_posterior0(p, F, e) for p in coords])
  return log_prob

# we'll start at random locations
starting_guesses = np.random.rand(nwalkers, ndim)
starting_guesses[:, 0] *= 2000  # start mu between 0 and 2000
starting_guesses[:, 1] *= 30    # start sigma between 0 and 20


## Rozkłady warunkowe i MC

Przypominam (np. *Konstrukcje II:* *1.3.5, 2.1.4*), że dla dowolnych zmiennych losowych $X\colon \Omega\to \R^p,$ $Z\colon\Omega\to\R^m\ $ **warunkowym rozkładem** zmiennej $X$ względem $Z$ nazywamy każdą (istnieją i są równe pw.) rodzinę borelowskich miar probabilistycznych
$$
\PP_{X|Z} = \left(\PP_{X|Z=z}\right)_{z\in\R^m}\colon\ \R^m\!\times\fB_{\R^p}\to[0,1]
$$
takich, że
$$
\PP\{X\in A, Z\in B\}=\int_B \PP_{X|Z=z}(A)\,\PP_Z(dz)\ \text{ dla }\ (A,B)\in\fB_{\R^p}\times\fB_{\R^m}.
$$
Wówczas także
$$
\int_\Omega h(X,Z)\,d\PP =\int_{\R^m}\left(\int_{\R^p}h(x,z)\PP_{X|Z=z}(dx) \right)\PP_Z(dz) \quad\text{ oraz} \\
$$
$$
\int_{X^{-1}A\,\cap\, Z^{-1}B} h(X,Z)\,d\PP =\int_{B}\left(\int_{A}h(x,z)\PP_{X|Z=z}(dx) \right)\PP_Z(dz)
$$
dla dowolnej funkcji borelowskiej $h\colon\R^{m+p}\to\R$, dla której przynajmniej jedna ze stron ma skończony sens.

Ogólna koncepcja **łańcuchów Markowa (MC)** zakłada, że ciąg wektorowych zmiennych losowych $\ X_n\colon \cX\to\R^p,\,n\ge 0\,,$   jest powiązany iteracyjną zależnością postaci
$$\begin{align}
\PP(X_{k+1}\in C|X_0=x_0,\ldots,X_k=x_k)&=\PP(X_{k+1}\in C|X_k=x_k) \\
&=\PP_{X_{k+1}|X_k=x_k}(C)\\
&= K(x_k,C)
\end{align}$$
dla $k\ge 0$, $C\in\fB_{\R^p}$ i dowolnych wartości $x_0,\ldots,x_k\in\R^p,$ przy czym $$\PP_{X_{k+1}|X_k}=K\colon\ \R^p\!\times\fB_{\R^p}\to[0,1]$$
jest niezależnym od $k,$ ustalonym rozkładem warunkowym zwanym **jądrem przejścia** (ang. transition kernel).

>Jeśli zmienne tworzące łańcuch są *dyskretne*, ich zbiór wartości jest co najwyżej przeliczalną *przestrzenią stanów*, a jądro przejścia opisuje macierz stochastyczna $[\PP(X_{n+1}=j|X_n=i)]$ zawierająca prawdopodobieństwa przejścia $i\leadsto j$  (od stanu $i$ do stanu $j$).


Oznaczmy $\PP_{X_0}=\mu.$ Ze względu na tożsamości
$$\begin{align}
\PP\{(X_0,X_1)\in A_0\times A_1\}&=\int_{A_0}\PP_{X_1|X_0=x_0}\!(A_1)\,\PP_{X_0}(dx_0) \\
&=\int_{A_0}K(x_0,A_1)\,\mu(dx_0), \\
\PP\{(X_0,X_1,X_2)\in A_0\times A_1\times A_2\}&=\int_{A_0\times A_1}\PP_{X_2|(X_0,X_1)}\!(A_2)\,d\PP_{(X_0,X_1)} \\
&=\int_{A_0\times A_1}\PP_{X_2|X_1=x_1}(A_2)\,\PP_{(X_0,X_1)}(dx_0,dx_1) \\
&=\int_{A_0}\int_{A_1}K(x_1,A_2)K(x_0,dx_1\!)\,\mu(dx_0) \\
\PP\{(X_0,\ldots,X_3)\in A_0\times \cdots\times A_3\}&= \int_{A_0}\ldots\int_{A_2}K(x_2,A_3)K(x_1,dx_2\!)K(x_0,dx_1\!)\,\mu(dx_0)
\end{align}$$
(... itd.) rozkład *startowy* $\mu$ wraz z jądrem przejścia $K$ wyznaczają jednoznacznie rozkład prawdopodobieństwa całego łańcucha Markowa -- w przestrzeni $(\R^p)^\infty$ -- oznaczany zazwyczaj jako $\PP_\mu.$

> W szczególności, jeśli przyjąć $\mu=\delta_x$ (czyli $X_0\equiv x\in\R^p$ jest stałe), rozważamy łańcuchy Markowa o zadanym *punkcie startowym* $x$ i rozkładach $\PP_x$.  

Istnienie podanych wyżej całek, a więc istnienie i jednoznaczny opis rozkładów $\PP_{(X_0,\ldots,X_n)},$ $n\in\N,$ nie wymaga dodatkowych założeń - własności rozkładu $\PP_\mu$ wynikają z przyjętych jako punkt wyjścia własności rozkładów $\mu$ i $K$.   
Każdy z rozkładów $\PP_{X_n}$ jest rozkładem brzegowym dla $\PP_\mu$ i wyraża się wzorem $\PP\{X_n\in A\} = \int K^n(x,A)\,\mu(dx),$ gdzie $K^1\equiv K$, a jądro *iterowane* $K^n$ jest splotem
$$
K^n(x_0,A) = \int\cdots\int K(x_{n-1},A)K(x_{n-2},dx_{n-1})\cdots K(x_0,dx_1)
$$
dla $n\ge 2.$
> Trywialny przykład jądra $\ K(x,\cdot)\equiv\kappa\ $ - stałego, niezależnego od $x$, implikuje tożsamości  $\ K^n=\kappa=\PP_{X_n}\ $ dla $n\ge 1,$ co podkreśla odmienny charakter rozkładu startowego $\mu.$


Zaawansowana teoria MC's dowodzi, że przy relatywnie niezbyt wymagających założeniach dotyczących jądra przejścia:
* istnieje dokładnie jedna *stacjonarna* miara probabilistyczna $\pi$ taka, że dla $\mu\equiv\pi$ wszystkie kolejne zmienne łańcucha mają ten sam rozkład $\PP_{X_n}\equiv\mu$
* niezależnie od wyboru początkowego rozkładu $\mu$, a wiec także - niezależnie od wyboru punktu startowego łańcucha, ciąg rozkładów $\PP_{X_n}$ jest zbieżny (w sensie całkowitej wariancji) do rozkładu stacjonarnego $\pi$.

> Całkowitą wariację różnicy miar definiuje wzór
$$\|\mu_1-\mu_2\|_{TV} = \sup_A|\mu_1(A)-\mu_2(A)|
$$

**Stacjonarność rozkładu** $\pi$ charakteryzuje pojedynczy warunek
$$
\int K(x,A)\,\pi(dx) = \pi(A)\quad\text{ dla } A\in \fB_{\R^p}.
$$


### Odwracanie kota ogonem
Skoro skomplikowana i dość ogólna konstrukcja, jaką jest MC (łańcuch Markowa), generuje zbieżny ciąg rozkładów, a graniczna miara $\pi$ jest jednoznacznie scharakteryzowana przez warunek stacjonarności, spróbujmy odwrócić kierunek rozumowania. Jeśli zatem *dana jest* miara probabilistyczna $\pi$ na $\cX\subset\R^p$, a przynajmniej jej gęstość $f$, to ewentualna konstrukcja *jądra przejścia* $K$ związanego z $f$ warunkiem stacjonarności
$$ \int K(x,A)f(x)\,dx = \int_A f(x)\,dx\quad\text{ dla }A\in\fB_\cX
$$      
pozwala traktować $\pi$ jako granicę ciągu rozkładów $\PP_{X_n}$ pewnego łańcucha Markowa.   
Warunek stacjonarności może być *łatwy do sprawdzenia*, jeśli $K$ w jawny sposób zależy od  funkcji gęstości $f$ - w takim przypadku iteracyjnie tworzone próbki rozkładów $\PP_{X_n}$ powinny być(?) zbieżne(?) do próbki rozkładu $\pi$.  

Jest to wprawdzie pewna komplikacja, ale także szansa na uzyskanie przybliżonego rozkładu i - w szczególności - możliwość przybliżonego, numerycznego wyznaczania związanych z $\pi$ wartości oczekiwanych (całek względem miary).



####  Algorytm Nicolasa Metropolis (1953) ... i Wilfreda K. Hastingsa (1970) jako **przykład**
Podstawowa koncepcja generowania próby


In [ ]:
def mh_sampler(x0, lnprob_fn, prop_fn, prop_fn_kwargs={}, iterations=100000):
    """Simple metropolis hastings sampler.

    :param x0: Initial array of parameters.
    :param lnprob_fn: Function to compute log-posterior.
    :param prop_fn: Function to perform jumps.
    :param prop_fn_kwargs: Keyword arguments for proposal function
    :param iterations: Number of iterations to run sampler. Default=100000

    :returns:
        (chain, acceptance, lnprob) tuple of parameter chain , acceptance rate
        and log-posterior chain.
    """

    # number of dimensions
    ndim = len(x0)

    # initialize chain, acceptance rate and lnprob
    chain = np.zeros((iterations, ndim))
    lnprob = np.zeros(iterations)
    accept_rate = np.zeros(iterations)

    # first samples
    chain[0] = x0
    lnprob0 = lnprob_fn(x0)
    lnprob[0] = lnprob0

    # start loop
    naccept = 0
    for ii in range(1, iterations):

        # propose
        x_star, factor = prop_fn(x0, **prop_fn_kwargs)

        # draw random uniform number
        u = np.random.uniform(0, 1)

        # compute hastings ratio
        lnprob_star = lnprob_fn(x_star)
        H = np.exp(lnprob_star - lnprob0) * factor

        # accept/reject step (update acceptance counter)
        if u < H:
            x0 = x_star
            lnprob0 = lnprob_star
            naccept += 1

        # update chain
        chain[ii] = x0
        lnprob[ii] = lnprob0
        accept_rate[ii] = naccept / ii

    return chain, accept_rate, lnprob

In [ ]:
def gaussian_proposal(x, sigma=0.1):
    """
    Gaussian proposal distribution.

    Draw new parameters from Gaussian distribution with
    mean at current position and standard deviation sigma.

    Since the mean is the current position and the standard
    deviation is fixed. This proposal is symmetric so the ratio
    of proposal densities is 1.

    :param x: Parameter array
    :param sigma:
        Standard deviation of Gaussian distribution. Can be scalar
        or vector of length(x)

    :returns: (new parameters, ratio of proposal densities)
    """

    # Draw x_star
    x_star = x + np.random.randn(len(x)) * sigma

    # proposal ratio factor is 1 since jump is symmetric
    qxx = 1

    return (x_star, qxx)

# x0 = np.random.randn(1)
chain, ar, lnprob = mh_sampler(starting_guesses, log_posterior0, gaussian_proposal,
                                prop_fn_kwargs={'sigma':sigma})
